In this notebook different quantisation methods and distance metrics for Facial Recognition will be compared both on accuracy and execution time. 

The Quantisation methods include:
- Scalar Quantisation
- TensorFlow Quantisation

The distance metrics include:
- Cosine Similarity
- Euclidean Distance

Below are the necassary import to run the code.

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # suppress tensorflow warnings https://stackoverflow.com/a/40871012
from deepface import DeepFace
import subprocess
import numpy as np
from decimal import Decimal # for proper rounding
import random
import time
import pandas as pd
from datetime import datetime
import struct
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import statistics
import accuracy as ac
import pickle
import quantisations as qt


import basics as bs


# CONSTANTS
EXECUTABLE_PATH = "ABY/build/bin"
INPUT_FILE_NAME = "input_vecs.txt"
EXECUTABLE_NAME_SCENARIO = 'cos_dist_copy'
CMD_SCENARIO = f"./{EXECUTABLE_NAME_SCENARIO} -r 1 -f {INPUT_FILE_NAME} & (./{EXECUTABLE_NAME_SCENARIO} -r 0 -f {INPUT_FILE_NAME} 2>&1 > /dev/null)"

# random number generator
rng = np.random.default_rng()

Below are two functions to compare Facenet and Sface accuracy. One for Euclidean Distance and one for Cosine Similarity. The code to create a visual representation for this comparison is also included.

In [2]:
###### GENERATING THE GLOBAL PAIRS. 
######## we only need to run this once and then we can have the file and use pairs as the list of embeddings 
########## uncomment to use for first them, then use the next cell!

##########!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!##########
## IF U GENERATE A NEW FILE, CHANGE THE NAME!!!! THE EMBEDINGPAIRS.PKL IS THE WORKING ONE AND THE ONE USED!!!!!!!!!!!!!!!!!!!##########
##########!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!##########

# #
# # Generate pairs globally
# def generate_pairs(m):
#     pairs = []
#     for _ in range(m):
#         print(_)
#         n = random.choice([True, False])
#         imga, imgb = bs.get_two_random_images(same_person=n)
#         pairs.append((imga, imgb,n))
#     return pairs

# m = 2000
# pairs = generate_pairs(m)

#File path
file_path = 'embedingpairs.pkl'

# # Delete the file if it exists
# if os.path.exists(file_path):
#     os.remove(file_path)

# # Save pairs to a new file
# with open(file_path, 'wb') as file:
#     pickle.dump(pairs, file)

In [3]:
with open(file_path, 'rb') as file:
    pairs = pickle.load(file)

In [4]:
# Running the comparison functions

counters_euc,times_euc = ac.compare_accuracies_euc(pairs)


In [5]:
counters_cos,times_cos = ac.compare_accuracies_cos(pairs)

In [6]:
counters_euc

{'scalar_quantisation_max': {'correct_tensor_facenet': 555,
  'wrong_tensor_facenet': 445,
  'correct_scalar_facenet': 555,
  'wrong_scalar_facenet': 445,
  'correct_noquant_facenet': 768,
  'wrong_noquant_facenet': 232,
  'correct_tensor_sface': 555,
  'wrong_tensor_sface': 445,
  'correct_scalar_sface': 555,
  'wrong_scalar_sface': 445,
  'correct_noquant_sface': 841,
  'wrong_noquant_sface': 159},
 'scalar_quantisation_percentile': {'correct_tensor_facenet': 555,
  'wrong_tensor_facenet': 445,
  'correct_scalar_facenet': 555,
  'wrong_scalar_facenet': 445,
  'correct_noquant_facenet': 768,
  'wrong_noquant_facenet': 232,
  'correct_tensor_sface': 555,
  'wrong_tensor_sface': 445,
  'correct_scalar_sface': 555,
  'wrong_scalar_sface': 445,
  'correct_noquant_sface': 841,
  'wrong_noquant_sface': 159}}

In [7]:
counters_cos

{'scalar_quantisation_max': {'correct_tensor_facenet': 458,
  'wrong_tensor_facenet': 542,
  'correct_scalar_facenet': 838,
  'wrong_scalar_facenet': 162,
  'correct_noquant_facenet': 837,
  'wrong_noquant_facenet': 163,
  'correct_tensor_sface': 458,
  'wrong_tensor_sface': 542,
  'correct_scalar_sface': 839,
  'wrong_scalar_sface': 161,
  'correct_noquant_sface': 838,
  'wrong_noquant_sface': 162},
 'scalar_quantisation_percentile': {'correct_tensor_facenet': 458,
  'wrong_tensor_facenet': 542,
  'correct_scalar_facenet': 812,
  'wrong_scalar_facenet': 188,
  'correct_noquant_facenet': 837,
  'wrong_noquant_facenet': 163,
  'correct_tensor_sface': 458,
  'wrong_tensor_sface': 542,
  'correct_scalar_sface': 819,
  'wrong_scalar_sface': 181,
  'correct_noquant_sface': 838,
  'wrong_noquant_sface': 162}}

In [8]:
m=1000
# Calculate and print the average execution times
print("Average Execution Times (in seconds):")
for name in times_euc.keys():
    avg_tensor_time_euc = times_euc[name]['tensor_time'] / m  # Assuming m=1000
    avg_scalar_time_euc = times_euc[name]['scalar_time'] / m
    
    avg_tensor_time_cos = times_cos[name]['tensor_time'] / m
    avg_scalar_time_cos = times_cos[name]['scalar_time'] / m

    print(f"\nQuantization Method: {name}")
    print(f"  Total Tensor Time (Euclidean): {times_euc[name]['tensor_time']} seconds")
    print(f"  Total Scalar Time (Euclidean): {times_euc[name]['scalar_time']} seconds")
    print(f"  Average Tensor Time (Euclidean): {avg_tensor_time_euc:.6f} seconds")
    print(f"  Average Scalar Time (Euclidean): {avg_scalar_time_euc:.6f} seconds")
    
    print(f"  Total Tensor Time (Cosine): {times_cos[name]['tensor_time']} seconds")
    print(f"  Total Scalar Time (Cosine): {times_cos[name]['scalar_time']} seconds")
    print(f"  Average Tensor Time (Cosine): {avg_tensor_time_cos:.6f} seconds")
    print(f"  Average Scalar Time (Cosine): {avg_scalar_time_cos:.6f} seconds")


Average Execution Times (in seconds):

Quantization Method: scalar_quantisation_max
  Total Tensor Time (Euclidean): 1.951538324356079 seconds
  Total Scalar Time (Euclidean): 0.20895147323608398 seconds
  Average Tensor Time (Euclidean): 0.001952 seconds
  Average Scalar Time (Euclidean): 0.000209 seconds
  Total Tensor Time (Cosine): 0.9779741764068604 seconds
  Total Scalar Time (Cosine): 0.1683039665222168 seconds
  Average Tensor Time (Cosine): 0.000978 seconds
  Average Scalar Time (Cosine): 0.000168 seconds

Quantization Method: scalar_quantisation_percentile
  Total Tensor Time (Euclidean): 1.951538324356079 seconds
  Total Scalar Time (Euclidean): 1.1559088230133057 seconds
  Average Tensor Time (Euclidean): 0.001952 seconds
  Average Scalar Time (Euclidean): 0.001156 seconds
  Total Tensor Time (Cosine): 0.9779741764068604 seconds
  Total Scalar Time (Cosine): 1.080245018005371 seconds
  Average Tensor Time (Cosine): 0.000978 seconds
  Average Scalar Time (Cosine): 0.001080 s

below will be the functions to compare the execution time of (Facenet, SFace) x (Euclidean, Cosine) X (no quantisation, Tensorflow, scalar)